In [2]:
from pyspark import SparkConf, SparkContext

#스파크 환경 설정 및 객체 생성

conf = SparkConf().setMaster("local").setAppName("241204_01.RDD_API")
spark = SparkContext(conf = conf).getOrCreate()

24/12/04 11:31:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

<SparkContext master=local appName=241204_01.RDD_API>

In [4]:
foods = spark.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [5]:
#collect() 모든 데이터 메모리에 올림 
#각각의 요소는 value임 
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [6]:
#countByValue() : value를 기준으로 개수 세기 -> 음식별 개수 세기 
#driver가 해당 명령을 excuter에게 보냄 
#하나의 클러스터에서 데이터 처리를 함 

foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [7]:
#상위 3개 데이터 추출 
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [16]:
#맨 처음 1개 데이터 추출 
foods.first()

'짜장면'

In [15]:
#RDD 전체 개수
foods.count()

12

In [14]:
#중복 제거한 RDD 개수 
#Transformation 연산 
fd_dist = foods.distinct()
fd_dist

PythonRDD[20] at RDD at PythonRDD.scala:53

In [17]:
#Action연산 : Result를 반환하는 연산 
fd_dist.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [18]:
#워커노드에서 실행하는 기능
#loop를 돌면서 하나씩 실행하고 싶을 때 사용 
#여러 워커노드가 있을 때 각각의 워커노드 내의 데이터를 체크할 때 사용 

foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [20]:
foods.foreach(lambda x : x)

## Narrow Operaion 1:1 연산

- filter(), map(), flatMap(), sample(), union()

In [22]:
sample_rdd = spark.parallelize( [1,2,3,4,5])

In [23]:
sample_rdd2 = sample_rdd.map(lambda x : x+2)

In [24]:
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [40]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [41]:
moviesRDD = spark.parallelize(movies)

In [42]:
MapMovies = moviesRDD.map(lambda x : x.split(" "))

In [43]:
MapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [44]:
flatMapMovies = moviesRDD.flatMap(lambda x:x.split(" "))
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [47]:
filteredMovies = flatMapMovies.filter(lambda x : x != "매트릭스")
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [48]:
#집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [49]:
#intersection : 교집합
num1.intersection(num2).collect()

[4, 5]

In [50]:
#union : 합집합
num1.union(num2).collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [52]:
#subtract : 차집합
num1.subtract(num2).collect()

[2, 1, 3]

#### sample(withReplacement, fraction , seed)  

일부를 샘플링하여 추출

- withReplacement 복원(중복허용 = True) , 비복원(중복불가 = False)
- fraction 기댓값
- seed 난수 추출을 위한 시드값 - 재현 가능

In [53]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [61]:
numlist.sample(True, 0.5).collect()

[5, 4, 8, 10, 10]

In [57]:
numlist.sample(True, 0.3).collect()

[1, 3, 6, 8, 9]

In [63]:
numlist.sample(False, 0.3).collect()

[4, 7]

In [66]:
numlist.sample(False, 0.3, seed=1).collect() #값 고정 

[5, 8]

## Wide Transformation

- groupBy()
- reduce()

In [67]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [69]:
foodsGroup = foods.groupBy(lambda x : x[0])
res = foodsGroup.collect()

In [70]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [71]:
spark.stop()